<a href="https://colab.research.google.com/github/korayaslaan/artificial_intelligence/blob/main/Sputniknews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta
import os
import csv
import concurrent
import multiprocessing
from multiprocessing import pool
import io
from pprint import pprint

In [128]:
def dateCreator():
  science = []
  tech = []
  #https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-01&date_to=2021-12-01   2005-02-01
  ser_date = pd.Series(pd.date_range('20050201', periods=6180))
  linkscience = "https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from="
  linktech = "https://sputniknews.com/services/search/getmore/?query=tech&tags_limit=20&tags=0&interval=period&date_from="
  link2 = "&date_to="
  a = 6120
  b = 6179  #6179 son 0 dan başlicak
  for j in range(a,b):
    dateEnd  = ser_date[j].strftime("%Y-%m-%d")
    science.append("{}{}{}{}".format(linkscience,dateEnd,link2,dateEnd))

  for j in range(a,b):#6179 son 0 dan başlicak
    dateEnd  = ser_date[j].strftime("%Y-%m-%d")
    tech.append("{}{}{}{}".format(linktech,dateEnd,link2,dateEnd))
    
  return science,tech 

In [129]:
def get_link_tech(i):
     count = 0
     r = requests.get(i)   
     soup = BeautifulSoup(r.content, 'html5lib')
     for a in soup.find_all('a', href=True):
       count+=1
       link = a['href']
       result = link.endswith("ml")
       if(result == True):
         if(count % 2 == 0):
           url  =   "https://sputniknews.com/"
           linkson = "{}{}".format(url,link)
           with open("TechLink.txt", 'a') as file: 
               file.write(linkson+'\n')

In [130]:
def get_link_science(i):
     count = 0
     r = requests.get(i)   
     soup = BeautifulSoup(r.content, 'html5lib')
     for a in soup.find_all('a', href=True):
       count+=1
       link = a['href']
       result = link.endswith("ml")
       if(result == True):
         if(count % 2 == 0):
           url  =   "https://sputniknews.com/"
           linkson = "{}{}".format(url,link)
           with open("ScienceLink.txt", 'a') as file: 
               file.write(linkson+'\n')

In [131]:
def creator_tech(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content = ""
    for wrapper in soup.find_all('div', {"data-type":"text"}):
       content = content + wrapper.text
    category = "tech"
    date = soup.find("a", attrs = {"class":"convert-date"}).getText()
    date = date.split()
    date =  date[2]
    w_data="{};{};{};{};{}".format(url,date,category,title,content)
    with open("Creator2.txt", 'a') as file: 
      file.write(w_data+'\n')

In [132]:
def creator_science(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content = ""
    for wrapper in soup.find_all('div', {"data-type":"text"}):
       content = content + wrapper.text
    category = "science"
    date = soup.find("a", attrs = {"class":"convert-date"}).getText()
    date = date.split()
    date =  date[2]
    w_data="{};{};{};{};{}".format(url,date,category,title,content)
    with open("Creator2.txt", 'a') as file: 
        file.write(w_data+'\n')

In [133]:
creator_science("https://sputniknews.com/20211231/so-that-was-2021-trump-leaves-white-house-taliban-retakes-kabul-but-mostly-that-infernal-virus-1091725615.html")

In [134]:
science,tech= dateCreator()

with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link_science,i.strip()) for i in science]

with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link_tech,i.strip()) for i in tech]

In [135]:
SputnikNewsScience=[]
with open("ScienceLink.txt",'r',newline='') as f:
  for i in f.readlines():    
    SputnikNewsScience.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator_science,i.strip()) for i in SputnikNewsScience]

print(time.time()-t1)

SputnikNewsTech=[]
with open("TechLink.txt",'r',newline='') as f:
  for i in f.readlines():    
    SputnikNewsTech.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator_tech,i.strip()) for i in SputnikNewsTech]

print(time.time()-t1)

132.4708218574524
171.53177976608276
